# 常见场景

## 意图识别

In [3]:
deployment = "gpt-4o"
model = "gpt-4o"

In [21]:

import os
from openai import AzureOpenAI
from openai.types.chat import ChatCompletion

client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE")
)


def chat(message: str) -> ChatCompletion:
    return client.chat.completions.create(
        model=deployment,
        messages=message,
        temperature=0.8,
        max_tokens=1000
    )


messages = [
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "提醒我明早8点有会议"},
    ],
    [
        {"role": "system", "content": "Recognize intent from user's input"},
        {"role": "user", "content": "订明天早5点北京到上海的飞机"},
    ]
]

for msg in messages:
    res = chat(msg)
    print(res.choices[0].message.content, res.usage.total_tokens)


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}